# Python для анализа данных

## Pandas (join и merge)

*Ян Пиле, НИУ ВШЭ*

Давайте представим, что мы аналитики в компании, которая производит и продает скрепки. Нам нужно составить несколько отчетов для отдела продаж, чтобы посмотреть, как у них дела. Данные лежат в следующих словарях:

In [16]:
import numpy as np
import pandas as pd
# Dataframe of number of sales made by an employee
sales = {'Tony': 103,
         'Sally': 202,
         'Randy': 380,
         'Ellen': 101,
         'Fred': 82
        }
# Dataframe of all employees and the region they work in
region = {'Tony': 'West',
          'Sally': 'South',
          'Carl': 'West',
          'Archie': 'North',
          'Randy': 'East',
          'Ellen': 'South',
          'Fred': np.nan,
          'Mo': 'East',
          'HanWei': np.nan,
         }

Мы можем создать два отдельных dataframe из словарей 

In [17]:
# Make dataframes
sales_df = pd.DataFrame.from_dict(sales, 
                                  orient='index', 
                                  columns=['sales'])
region_df = pd.DataFrame.from_dict(region, 
                                   orient='index', 
                                   columns=['region'])

In [18]:
sales_df

,sales
Tony,103
Sally,202
Randy,380
Ellen,101
Fred,82


In [19]:
region_df

,region
Tony,West
Sally,South
Carl,West
Archie,North
Randy,East
Ellen,South
Fred,NaN
Mo,East
HanWei,NaN


Теперь давайте объединим все наши данные в один Датафрейм. Но как нам это сделать?.

### Join
Давайте начнем с join, потому что он самый простой. У датафреймов данных есть параметр index. Это ключ вашей таблицы, и если мы знаем индекс, то мы можем легко получить строку, содержащую наши данные, используя .loc. Если вы напечатаете свой датафрейм, вы увидите индекс в крайнем левом столбце. Еще его можно получить, напрямую использовав .index:

In [20]:
sales_df.index

Index(['Tony', 'Sally', 'Randy', 'Ellen', 'Fred'], dtype='object')

Значит индекс в sales_df - это имя продавца. Кстати, в отличие от первичного ключа таблицы SQL, индекс датафрейма не обязательно должен быть уникальным. Но уникальный индекс делает нашу жизнь проще, а время, необходимое для поиска в нашем датафрейме, короче. Учитывая индекс, мы можем найти данные строки так:

In [21]:
sales_df.loc['Tony']

sales    103
Name: Tony, dtype: int64

Вернемся к join'ам. Метод join берет два датафрейма и соединяет их по индексам (технически вы можете выбрать столбец для объединения для левого датафрейма). Давайте посмотрим, что происходит, если мы объединим(aka сджойним) наши два датафрейма с помощью метода join:

In [22]:
joined_df = region_df.join(sales_df, how='left')
joined_df

,region,sales
Tony,West,103.0
Sally,South,202.0
Carl,West,NaN
Archie,North,NaN
Randy,East,380.0
Ellen,South,101.0
Fred,NaN,82.0
Mo,East,NaN
HanWei,NaN,NaN


Метод join использует индекс или указанный столбец из левого датафрейма в качестве ключа джойна. Таким образом, столбец, по которому мы джойним левый датафрейм, не обязательно должен быть его индексом. Но вот для правого датафрейма ключ джойна должен быть его индексом ОБЯЗАТЕЛЬНО. 

Лично мне проще воспринимать метод join как объединение на основе индекса и использовать merge, если не хочется привязываться к индексам.

В объединенном датафрейме есть несколько NaN. Так произошло , потому что не у всех сотрудников были продажи. Те, у кого не было продаж, отсутствуют в sales_df, но мы по-прежнему отображаем их, потому что мы выполнили left join (указав «how = left»), которое возвращает все строки из левого датафрейма region_df, независимо от того, есть ли совпадение в правом. Если мы не хотим отображать какие-либо NaN в нашем результате соединения, мы вместо этого можем сделать inner join (указав «how = inner»).

### Merge

На базовом уровне Merge делает более или менее то же самое, что и join. Оба метода используются для объединения двух датафреймов, но merge является более универсальным за счет более подробного описания входных данных. Давайте посмотрим, как мы можем создать тот же объединенный датафрейм с помощью merge:

In [28]:
sales_df

,sales
Tony,103
Sally,202
Randy,380
Ellen,101
Fred,82


In [29]:
region_df

,region
Tony,West
Sally,South
Carl,West
Archie,North
Randy,East
Ellen,South
Fred,NaN
Mo,East
HanWei,NaN


In [30]:
joined_df_merge = region_df.merge(sales_df, how='left', 
                                      left_index=True,
                                      right_index=True)
print(joined_df_merge)

       region  sales
Tony     West  103.0
Sally   South  202.0
Carl     West    NaN
Archie  North    NaN
Randy    East  380.0
Ellen   South  101.0
Fred      NaN   82.0
Mo       East    NaN
HanWei    NaN    NaN


In [31]:
joined_df_merge

,region,sales
Tony,West,103.0
Sally,South,202.0
Carl,West,NaN
Archie,North,NaN
Randy,East,380.0
Ellen,South,101.0
Fred,NaN,82.0
Mo,East,NaN
HanWei,NaN,NaN


In [32]:
joined_df_merge.reset_index(inplace=True)

In [33]:
joined_df_merge

,index,region,sales
0,Tony,West,103.0
1,Sally,South,202.0
2,Carl,West,NaN
3,Archie,North,NaN
4,Randy,East,380.0
5,Ellen,South,101.0
6,Fred,NaN,82.0
7,Mo,East,NaN
8,HanWei,NaN,NaN


In [36]:
joined_df_merge['name'] = joined_df_merge['index']
joined_df_merge

,index,region,sales,name
0,Tony,West,103.0,Tony
1,Sally,South,202.0,Sally
2,Carl,West,NaN,Carl
3,Archie,North,NaN,Archie
4,Randy,East,380.0,Randy
5,Ellen,South,101.0,Ellen
6,Fred,NaN,82.0,Fred
7,Mo,East,NaN,Mo
8,HanWei,NaN,NaN,HanWei


In [38]:
joined_df_merge.rename(columns={'index':'Name'})

,Name,region,sales,name
0,Tony,West,103.0,Tony
1,Sally,South,202.0,Sally
2,Carl,West,NaN,Carl
3,Archie,North,NaN,Archie
4,Randy,East,380.0,Randy
5,Ellen,South,101.0,Ellen
6,Fred,NaN,82.0,Fred
7,Mo,East,NaN,Mo
8,HanWei,NaN,NaN,HanWei
